# Zero shot feelings and needs guesses

In [ ]:
import importlib

import numpy as np
import tensorflow as tf

from transformers import pipeline

from howie import feelings, needs

In [ ]:
#boiler plate for limiting allocated GPU memory
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


# Guessing emotions and needs

In [ ]:
importlib.reload(feelings)
importlib.reload(needs)

In [ ]:
classifier = pipeline("zero-shot-classification", device=0)

In [ ]:
sequence = "I haven't eaten all day."
hypothesis_template = "I feel {}."
results = classifier(sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
sequence = "I aced the interview and got the job."
hypothesis_template = "I feel {}."
results = classifier(sequence, feelings.nuanced_feelings, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
sequence = "I aced the interview and got the job."
hypothesis_template = "I need {}."
results = classifier(sequence, needs.needs, hypothesis_template=hypothesis_template,multi_label=True)
results['labels'][:5]

In [ ]:
num_feelings = 5
sequence = "You should eat more brussel sprouts."
#hypothesis_template = "I feel {}."
hypothesis_template = "Am I feeling {}?"
feelings_results = classifier(sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
feelings_results['labels'][:5]
for feeling in feelings_results['labels'][:num_feelings]:
    hypothesis_template = "Am I feeling " +feeling+ " because I have a need for {}?"
    need_results = classifier(sequence, needs.needs, hypothesis_template=hypothesis_template,multi_label=True)
    print(feeling, need_results['labels'][:3])

In [ ]:
num_feelings = 5
num_needs = 5
original_sequence = "Talking with human resources people is really difficult and I'm afraid of being jugdged."
#hypothesis_template = "I feel {}."
hypothesis_template = "Am I feeling {}?"
feelings_results = classifier(original_sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
feelings_results['labels'][:5]
used_needs = set()
for feeling in feelings_results['labels'][:num_feelings]:
    sequence = "When I hear "+original_sequence+" do I feel "+feeling
    hypothesis_template = "because I have a need for {}?"
    need_results = classifier(sequence, needs.needs, hypothesis_template=hypothesis_template,multi_label=True)
    used_needs = used_needs.union(need_results['labels'][:num_needs])
    #set(used_needs)
    print("Am I feeling "+ feeling+" because I have a need for ",end="")
    print(*need_results['labels'][:3],sep=", ",end="")
    print(", and " + need_results['labels'][3]+"?",sep=", ")

In [ ]:
num_feelings = 5
num_needs = 5
original_sequence = "The little pup came over and started biting my sleeve for food."
#hypothesis_template = "I feel {}."
hypothesis_template = "Am I feeling {}?"
feelings_results = classifier(original_sequence, feelings.base_feelings, hypothesis_template=hypothesis_template,multi_label=True)
feelings_results['labels'][:5]
used_needs = set()
updated_needs = needs.needs
print("When I hear "+ original_sequence + " do I feel ")
for idx, feeling in enumerate(feelings_results['labels'][:num_feelings]):
    sequence = "When I hear "+original_sequence+" do I feel "+feeling
    hypothesis_template = "because I have a need for {}?"
    need_results = classifier(sequence, list(updated_needs), hypothesis_template=hypothesis_template,multi_label=True)
    used_needs = used_needs.union(need_results['labels'][:num_needs])
    updated_needs = set(needs.needs).difference(used_needs)
    print(idx, feeling+" because I have a need for ",end="")
    print(*need_results['labels'][:3],sep=", ",end="")
    print(", and " + need_results['labels'][3]+"?",sep=", ")